In [19]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/warehouse'

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .enableHiveSupport() \
    .getOrCreate()

In [20]:
spark.sql(
    """
    SHOW TABLES FROM tabd_db
    """
).show()

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
| tabd_db|distributiongloba...|      false|
| tabd_db|globalmobilepenet...|      false|
| tabd_db|globalsocialnetworks|      false|
| tabd_db| internetactivities1|      false|
| tabd_db| internetactivities2|      false|
| tabd_db|       internetusage|      false|
| tabd_db|numberofworldwide...|      false|
| tabd_db|p_dailyinternetus...|      false|
| tabd_db|p_enterprisesocia...|      false|
| tabd_db|p_globalsocialmed...|      false|
| tabd_db|p_globalsocialnet...|      false|
| tabd_db|p_individualsacti...|      false|
| tabd_db|     p_internetusage|      false|
| tabd_db|parentawarenessof...|      false|
| tabd_db|socialmediausagef...|      false|
| tabd_db|socialmediausebyp...|      false|
| tabd_db|socialmediausebyt...|      false|
| tabd_db|socialmediausersa...|      false|
| tabd_db|  socialnetworkusers|      false|
+--------+--------------------+-

In [21]:
spark.sql(
    """
    DROP TABLE IF EXISTS tabd_db.NumberOfWorldwideSocialMediaUse 
    """
)

spark.sql(
    """
    CREATE EXTERNAL TABLE tabd_db.NumberOfWorldwideSocialMediaUse  (
        region VARCHAR(45),
        social_media_users DOUBLE,
        year INT,
        mobile_penetration_percentage INT,
        social_media_users_percentage FLOAT
    )
    STORED AS PARQUET
    LOCATION 'hdfs://hdfs-nn:9000/warehouse/tabd.db/NumberOfWorldwideSocialMediaUse/'
    """
)

DataFrame[]

In [22]:
spark.sql(
    """
    SHOW TABLES FROM tabd_db
    """
).show()

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
| tabd_db|distributiongloba...|      false|
| tabd_db|globalmobilepenet...|      false|
| tabd_db|globalsocialnetworks|      false|
| tabd_db| internetactivities1|      false|
| tabd_db| internetactivities2|      false|
| tabd_db|       internetusage|      false|
| tabd_db|numberofworldwide...|      false|
| tabd_db|p_dailyinternetus...|      false|
| tabd_db|p_enterprisesocia...|      false|
| tabd_db|p_globalsocialmed...|      false|
| tabd_db|p_globalsocialnet...|      false|
| tabd_db|p_individualsacti...|      false|
| tabd_db|     p_internetusage|      false|
| tabd_db|parentawarenessof...|      false|
| tabd_db|socialmediausagef...|      false|
| tabd_db|socialmediausebyp...|      false|
| tabd_db|socialmediausebyt...|      false|
| tabd_db|socialmediausersa...|      false|
| tabd_db|  socialnetworkusers|      false|
+--------+--------------------+-

In [23]:
# Let's look into HDFS
spark.sql(
    """
    SELECT *
    FROM tabd_db.NumberOfWorldwideSocialMediaUse
    """
).show()

+------+------------------+----+-----------------------------+-----------------------------+
|region|social_media_users|year|mobile_penetration_percentage|social_media_users_percentage|
+------+------------------+----+-----------------------------+-----------------------------+
+------+------------------+----+-----------------------------+-----------------------------+



In [24]:
spark.sql(
    """
    DESCRIBE FORMATTED tabd_db.NumberOfWorldwideSocialMediaUse
    """
).toPandas()

,col_name,data_type,comment
0,region,varchar(45),None
1,social_media_users,double,None
2,year,int,None
3,mobile_penetration_percentage,int,None
4,social_media_users_percentage,float,None
5,,,
6,# Detailed Table Information,,
7,Database,tabd_db,
8,Table,numberofworldwidesocialmediause,
9,Owner,jovyan,


In [25]:
# Let's put the files into HDFS

spark.sql(
    """
    SELECT *
    FROM tabd_db.NumberOfWorldwideSocialMediaUse
    """
).toPandas()

,region,social_media_users,year,mobile_penetration_percentage,social_media_users_percentage


In [26]:
# recover partitions is needed so that the Hive Metastore (Catalog)
# is updated. Otherwise Hive and the querying engines do not know
# that there are new parittions in the partitioned table.

spark.sql(
    """
    SELECT *
    FROM tabd_db.NumberOfWorldwideSocialMediaUse
    """
).show()

+------+------------------+----+-----------------------------+-----------------------------+
|region|social_media_users|year|mobile_penetration_percentage|social_media_users_percentage|
+------+------------------+----+-----------------------------+-----------------------------+
+------+------------------+----+-----------------------------+-----------------------------+



In [27]:
spark.stop()